In [108]:
import numpy as np
import torch
from molecular_mpns.config import data_dir
from molecular_mpns.proto_molecule import Molecule
from torch.autograd.functional import jacobian,hessian

In [109]:
def galerkin_coords(point,bins,lims):
    v,basis_edges_x,basis_edges_y = np.histogram2d(x = [point[0]],y = [point[1]],bins = [bins,bins], range = lims)
    return v,basis_edges_x,basis_edges_y
    
class PerronFrobenius2D:
    
    def __init__(self,n_basis,lims):
        
        self.n_basis = n_basis
        self.T = np.eye(n_basis**2)
        self.lims = lims
        
    
    def estimate(self,trajectory):
        
        for t in range(trajectory.shape[0]-1):
            current_state,next_state = galerkin_coords(trajectory[t],bins=self.n_basis,lims=self.lims)[0],galerkin_coords(trajectory[t+1],bins=self.n_basis,lims=self.lims)[0]
            current_ind,next_ind = np.argmax(current_state),np.argmax(next_state)
            self.T[:,current_ind] += next_state.flatten()
            
        self.T /= self.T.sum(axis = 0)
        
    def implied_timescales(self,dt,n_vals, eps = 1e-7):
        
        lamb = np.linalg.eigvals(self.T)
        idx = lamb.argsort()[::-1]   
        lamb = lamb[idx]
        lamb = lamb[0:n_vals]
        t_i = -(dt/np.log(lamb+eps))
        return t_i            

In [114]:
def dih_angle(x,first = True):
    
    x1,x2,x3,x4 = x[:-1] if first else x[1:] 
    u1 = x1-x2
    u2 = x2-x3
    u3 = x3-x4
    r2 = torch.norm(u2)
    
    y = torch.dot(u2,torch.cross(torch.cross(-u3,u2),torch.cross(u1,u2)))
    x = r2*torch.dot(torch.cross(u1,u2),torch.cross(-u3,u2))
    
    da = torch.atan2(y,x)
    
    return da

def langevian_gedmd_coarse_grain_transform(x,dV,beta):
    
    psidx,psiddx = jacobian(dih_angle,x),hessian(dih_angle,x)
    phidx,phiddx = jacobian(lambda x: dih_angle(x,first=False),x),hessian(lambda x: dih_angle(x,first=False),x)
    
    psidx,psiddx = psidx.detach().cpu().numpy(),psiddx.detach().cpu().numpy()
    phidx,phiddx = phidx.detach().cpu().numpy(),phiddx.detach().cpu().numpy()
    
    
    # compute b^{\xi}
    a = np.zeros((psiddx.shape))
    
    for i in range(a.shape[0]):
        for j in range(a.shape[1]):
            a[i,j,i,j] = (2/beta)
    
    bpsi,apsi = np.sum(dV*psidx),np.sum(a*psiddx)
    bphi,aphi = np.sum(dV*phidx),np.sum(a*phiddx)
    
    y = np.array([[bpsi+0.5*apsi],[bphi+0.5*aphi]])
    
    # compute a^{\xi}
    p,d = np.shape(x)
    sigma = np.sqrt(2/beta)*np.eye(int(p*d))
    xi = np.vstack([psidx.flatten(),phidx.flatten()])
    z = np.matmul(xi,sigma)
    
    return y,z
    

In [115]:
# set up molecular environment 
N = 5
beta = 15.0
kb = 1.0
rb = 2.0
ka = 1.0
ra = np.pi/2
kd = np.array([[0.02, 3], [0.02, 2]])
mol = Molecule(N, beta, kb, rb, ka, ra, kd)

# load data
path_to_data = str(data_dir)+'/proto_mol_traj.npy'
data = np.load(path_to_data)

x = data[2]
dV = mol._gradient(x)
beta = 4

x = torch.tensor(x)
y,z = langevian_gedmd_coarse_grain_transform(x,dV,beta)